# Twitter Bot Account Classification with ruCLIP


This notebook demonstrates how to use ruCLIP, for classifying Twitter accounts into binary classes based on multiple features.

We'll install dependencies, define our model configuration, train the model, and visualize performance metrics.

In [1]:
# Install required packages
!pip install ruclip==0.0.2
!pip install scikit-learn
!pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.1 MB/s eta 0:00:00
  Created wheel for youtokentome: filename=youtokentome-1.0.6-cp310-cp310-linux_x86_64.whl size=193386 sha256=65985855dc29d74264ac816345c767a2512fb42441fb7f85d6ab6c449e4d8bc3
  Stored in directory: /root/.cache/pip/wheels/df/85/f8/301d2ba45f43f30bed2fe413efa760bc726b8b660ed9c2900c
Successfully built youtokentome
  Attempting uninstall: more-itertools
    Found existing installation: more-itertools 10.3.0
    Uninstalling more-itertools-10.3.0:
      Successfully uninstalled more-itertools-10.3.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.25.1
    Uninstalling huggingface-hub-0.25.1:
      Successfully uninstalled huggingface-hub-0.25.1
ERROR: pip's depend

In [2]:
# Disable GPU

#import os

#os.environ["CUDA_VISIBLE_DEVICES"]="GPU-XXXXX" # MIG-GPU-XXXXX

#os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
# Install required packages
!pip install transformers
!pip install torch torchvision
!pip install scikit-learn
!pip install ruclip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.2.1
    Uninstalling huggingface-hub-0.2.1:
      Successfully uninstalled huggingface-hub-0.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ruclip 0.0.2 requires huggingface-hub==0.2.1, but you have huggingface-hub 0.26.2 which is incompatible.
  Using cached huggingface_hub-0.2.1-py3-none-any.whl.metadata (6.3 kB)
Using cached huggingface_hub-0.2.1-py3-none-any.whl (61 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.26.2
    Uninstalling huggingface-hub-0.26.2:
      Successfully uninstalled huggingface-hub-0.26.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

In [4]:
!pip install huggingface-hub==0.23.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.2.1
    Uninstalling huggingface-hub-0.2.1:
      Successfully uninstalled huggingface-hub-0.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ruclip 0.0.2 requires huggingface-hub==0.2.1, but you have huggingface-hub 0.23.3 which is incompatible.


In [5]:
# Import libraries
import torch
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from PIL import Image

In [6]:
import ruclip

# Load the CLIP model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
model, processor = ruclip.load("ruclip-vit-base-patch32-224", device=device)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:671: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


bpe.model:   0%|          | 0.00/748k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/ruclip/model.py:237: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(folder, 'pytorch_model.bin'),

In [7]:
# Load the Twitter dataset

dataset_df = pd.read_csv("/kaggle/input/twitter-human-bots-dataset/twitter_human_bots_dataset.csv")


# Append full path to image data

dataset_df['profile_image_path'] = '/kaggle/input/twitter-human-bots-dataset/' + dataset_df['profile_image_path']


# Display the first few rows of the dataset

dataset_df.head()

,Unnamed: 0,created_at,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,id,...,profile_image_url,screen_name,statuses_count,verified,average_tweets_per_day,account_age_days,account_type,profile_image_path,profile_background_image_path,split
0,0,2009-07-04 22:41:51,False,False,NaN,5007,102,168,False,53779179,...,https://pbs.twimg.com/profile_images/128969130...,paty_castroo,3974,False,0.978,4063,bot,NaN,NaN,0
1,1,2010-01-17 22:54:19,False,False,Television producer. Emmy Award winner. Disney...,1038,60,128,False,105916557,...,https://pbs.twimg.com/profile_images/632916759...,CBirckner,259,False,0.067,3866,human,/kaggle/input/twitter-human-bots-dataset/profi...,profile_images/4691ae58668d932bb7e7af22141aecf...,0
2,2,2012-03-01 06:05:32,False,False,NaN,2257,599,449,True,509788597,...,https://pbs.twimg.com/profile_images/128853844...,amf_jay,13627,False,4.406,3093,human,NaN,NaN,0
3,3,2009-09-01 04:52:30,False,False,NaN,6407,116,334,True,70601327,...,https://pbs.twimg.com/profile_images/977012905...,SaraCavolo,4432,False,1.107,4005,human,/kaggle/input/twitter-human-bots-dataset/profi...,profile_images/d1da9220e4aa376dff03b6f12765171...,0
4,4,2010-01-27 17:17:23,False,False,Productor de Televisión - Embajador de @Tienda...,20866,74448,18,True,108999927,...,https://pbs.twimg.com/profile_images/668449819...,DavidHenaoModel,15870,True,4.116,3856,human,/kaggle/input/twitter-human-bots-dataset/profi...,profile_images/c65bf625b066ef5f29e14cfcf1ee91f...,0


In [8]:
from tqdm import tqdm
import numpy as np

# Function to get CLIP embeddings with truncation for text, in batches with a progress bar
def get_clip_embeddings(texts, images, batch_size=32):
    all_embeddings = []
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Processing Embeddings"):
        # Process each batch
        batch_texts = texts[i:i + batch_size]
        batch_images = images[i:i + batch_size]
        
        # Preprocess text and images separately
        text_inputs = processor(text=batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=77)
        image_inputs = processor(images=batch_images, return_tensors="pt")

        # Move text and image inputs to the same device as the model
        text_inputs = {key: value.to(device) for key, value in text_inputs.items()}
        image_inputs = {key: value.to(device) for key, value in image_inputs.items()}

        # Get embeddings from CLIP for the batch using encode_text and encode_image
        text_embeddings = model.encode_text(text_inputs["input_ids"])
        image_embeddings = model.encode_image(image_inputs["pixel_values"])
        
        # Combine text and image embeddings (e.g., by summing or concatenating)
        batch_embeddings = text_embeddings + image_embeddings
        all_embeddings.append(batch_embeddings.cpu().detach().numpy())

    # Concatenate all batch embeddings into a single array
    return np.concatenate(all_embeddings, axis=0)

In [9]:
from PIL import Image, UnidentifiedImageError
from tqdm import tqdm

# Load and preprocess images with a progress bar, converting to RGBA if needed, resizing to 224x224
def load_images(image_paths):
    images = []
    for path in tqdm(image_paths, desc="Loading images"):
        try:
            # Check if the path is a valid string
            if isinstance(path, str):
                img = Image.open(path)
                
                # Convert palette images with transparency to RGBA
                if img.mode == "P" or img.mode == "LA":
                    img = img.convert("RGBA")
                
                # Resize to a standard size for CLIP (e.g., 224x224)
                img = img.resize((224, 224))
                images.append(img)
            else:
                # Append a blank image if the path is invalid
                images.append(Image.new("RGB", (224, 224), color="white"))
        
        # Handle errors (e.g., file not found or unreadable image)
        except (FileNotFoundError, UnidentifiedImageError):
            images.append(Image.new("RGB", (224, 224), color="white"))
            
    return images

# Clean and truncate text
dataset_df['description'] = dataset_df['description'].fillna("")
sample_texts = dataset_df['description'].astype(str).tolist()
sample_images = load_images(dataset_df['profile_image_path'])

Loading images: 100%|██████████| 37438/37438 [02:36<00:00, 239.45it/s]


In [10]:
# Now, generate the embeddings
embeddings = get_clip_embeddings(sample_texts, sample_images)

Processing Embeddings: 100%|██████████| 1170/1170 [02:33<00:00,  7.64it/s]


In [11]:
# Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(
    embeddings, dataset_df['account_type'], test_size=0.2, random_state=42
)

# Train a classifier (e.g., Logistic Regression)
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Predict and evaluate
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

         bot       0.67      0.57      0.62      2455
       human       0.81      0.86      0.83      5033

    accuracy                           0.77      7488
   macro avg       0.74      0.72      0.73      7488
weighted avg       0.76      0.77      0.76      7488

